### Natural Joins in Pandas and SQLite

This notebook investigates the use of a natural join in both SQL and Pandas, as well as the implications of the default behavior between the two approaches to joining tables. 

When a row is not identified uniquely by a single column value, you may need to JOIN ON more than one column value in your pandas or sql statement. If you'd like to use the full intersection of all columns (where you must have a matching value in every common column between two tables or dataframes), you can specify all columns or use the default behavior of pandas or the NATURAL JOIN statement in SQL. 

This workbook illustrates various options and approaches to managing JOINs with multiple common columns but no unique single column key in either table. 

#### Create data

We create two tables with three overlapping columns. Neither of these tables are uniquely identified by a single column. Note that the 4th column is identical for each of the two tables, but will have a different name in each table. 

In [23]:
t_1 = [
    ["A", "B", "C", "A"],
    ["A", "B", "D", "B"],
    ["B", "B", "C", "C"],
    ["A", "A", "C", "D"],
    ["B", "B", "D", "E"]
]

t_2 = [
    ["A", "B", "C", "A"],
    ["A", "B", "D", "B"],
    ["B", "B", "C", "C"],
    ["A", "A", "C", "D"],
    ["B", "B", "D", "E"]
]

In [24]:
import pandas as pd

In [25]:
df_1 = pd.DataFrame(t_1)
df_1.columns = ['C1', 'C2', 'C3', 'C4']

df_2 = pd.DataFrame(t_2)
df_2.columns = ['C1', 'C2', 'C3', 'C5']

In [26]:
df_1

,C1,C2,C3,C4
0,A,B,C,A
1,A,B,D,B
2,B,B,C,C
3,A,A,C,D
4,B,B,D,E


In [27]:
df_2

,C1,C2,C3,C5
0,A,B,C,A
1,A,B,D,B
2,B,B,C,C
3,A,A,C,D
4,B,B,D,E


### Pandas Merge

By default, a pandas merge operation will replicate the "natural join", using all common column headers as the "JOIN ON" condition from SQL. In other words, if you don't specify a join column, all common columns between the two tables must match.

In [28]:
df_1.merge(df_2)

,C1,C2,C3,C4,C5
0,A,B,C,A,A
1,A,B,D,B,B
2,B,B,C,C,C
3,A,A,C,D,D
4,B,B,D,E,E


If we specify a join column, we will get the cross product of both dataframe rows matching on this column. Because neither table is uniquely identified by this column, the result set will expand.

In [29]:
df_1.merge(df_2, on='C1')

,C1,C2_x,C3_x,C4,C2_y,C3_y,C5
0,A,B,C,A,B,C,A
1,A,B,C,A,B,D,B
2,A,B,C,A,A,C,D
3,A,B,D,B,B,C,A
4,A,B,D,B,B,D,B
5,A,B,D,B,A,C,D
6,A,A,C,D,B,C,A
7,A,A,C,D,B,D,B
8,A,A,C,D,A,C,D
9,B,B,C,C,B,C,C


If we specify all common columns in our query, we will get the same result as the default behavior. 

In [30]:
df_1.merge(df_2, on=['C1','C2','C3'])

,C1,C2,C3,C4,C5
0,A,B,C,A,A
1,A,B,D,B,B
2,B,B,C,C,C
3,A,A,C,D,D
4,B,B,D,E,E


### SQL JOIN

We can replicate the pandas work using SQL. 

In [31]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

A NATURAL JOIN in SQL will join on all common columns between two tables

In [32]:
pysqldf("""
SELECT
    *
FROM
    df_1
NATURAL JOIN
    df_2
""")

,C1,C2,C3,C4,C5
0,A,B,C,A,A
1,A,B,D,B,B
2,B,B,C,C,C
3,A,A,C,D,D
4,B,B,D,E,E


If we join only on the first column, we will get the cross product of both tables matching on this column, because C1 is not a unique identifier for each row.

In [33]:
pysqldf("""
SELECT
    *
FROM
    df_1
JOIN
    df_2
ON
    df_1.C1 = df_2.C1
""")

,C1,C2,C3,C4,C1,C2,C3,C5
0,A,B,C,A,A,A,C,D
1,A,B,C,A,A,B,C,A
2,A,B,C,A,A,B,D,B
3,A,B,D,B,A,A,C,D
4,A,B,D,B,A,B,C,A
5,A,B,D,B,A,B,D,B
6,B,B,C,C,B,B,C,C
7,B,B,C,C,B,B,D,E
8,A,A,C,D,A,A,C,D
9,A,A,C,D,A,B,C,A


If we explicitely JOIN on all common columns, we will replicate the NATURAL JOIN

In [34]:
pysqldf("""
SELECT
    *
FROM
    df_1
JOIN
    df_2
ON
    df_1.C1 = df_2.C1
    AND
    df_1.C2 = df_2.C2
    AND
    df_1.C3 = df_2.C3 
""")

,C1,C2,C3,C4,C1,C2,C3,C5
0,A,B,C,A,A,B,C,A
1,A,B,D,B,A,B,D,B
2,B,B,C,C,B,B,C,C
3,A,A,C,D,A,A,C,D
4,B,B,D,E,B,B,D,E


### Possible Pitfalls of NATURAL JOIN

If you read about this on the web, you'll probably find that some people are wary of using NATURAL JOIN, preferring instead to explicitely write out all matching columns - even if this means more typing. 

One worry is that you don't know what columns were used to JOIN two tables with a NATURAL JOIN unless you inspect the tables - you can no longer get this information from the SQL statement or code itself. This could lead to unexpected behavior - for example, you might miss a minor difference between two column names and believe you joined on all intersecting columns. 

In the above scenario, you may have noticed above that C4 and C5 have identical values, and you could use these two columns to JOIN the two tables.

In [35]:
pysqldf("""
SELECT
    *
FROM
    df_1
JOIN
    df_2
ON
    df_1.C4 = df_2.C5
""")

,C1,C2,C3,C4,C1,C2,C3,C5
0,A,B,C,A,A,B,C,A
1,A,B,D,B,A,B,D,B
2,B,B,C,C,B,B,C,C
3,A,A,C,D,A,A,C,D
4,B,B,D,E,B,B,D,E


However, because these columns have different names, they would not be included in the natural join. This scenario could also arise if table names were changed, or added, after you wrote your orignal queries. 

I'm not necessarily discouraging you from using a NATURAL join. If you're reading this, I'd guess you are probaably working with research data released by researchers, not IT data released by database administrators, and your queries are often very ad-hoc as you try to wrangle data into a usable form. So things like NATURAL JOINs, which may not be a big part of the usual SQL world, often come up in less structured and less predictable data engineering scenario. 

One last thing - the default behavior of SQL is completely different from the default behavior or pandas merge. By default, a SQL JOIN without an ON clause doesn't select the full intersection, it selects the full cross product of two tables, taking all combinations of the two sets. 

In [37]:
pysqldf("""
SELECT 
    *
FROM
    df_1
JOIN
    df_2
""")

,C1,C2,C3,C4,C1,C2,C3,C5
0,A,B,C,A,A,B,C,A
1,A,B,C,A,A,B,D,B
2,A,B,C,A,B,B,C,C
3,A,B,C,A,A,A,C,D
4,A,B,C,A,B,B,D,E
5,A,B,D,B,A,B,C,A
6,A,B,D,B,A,B,D,B
7,A,B,D,B,B,B,C,C
8,A,B,D,B,A,A,C,D
9,A,B,D,B,B,B,D,E
